In [1]:
%matplotlib inline
%run ../../path_import.py
from plots import *
from training_reparam import *
from eubo import *
from model_reparam import *

import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
data_path = "../gmm_dataset_c2k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
_, N, D = Data.shape
K = 3 ## number of clusters
## Model Parameters
MCMC_SIZE = 20
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32
## Training Parameters
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 5*1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-mcmc-v2-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)

DEVICE = torch.device('cuda:0')

In [3]:
# initialization
enc_z = Gibbs_z(K, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE)
if CUDA:
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

In [ ]:
# enc_eta.load_state_dict(torch.load('../weights/enc-eta-%s' + PATH))

In [ ]:
RESAMPLE = False
DETACH=True
train_cfz_reparam(Eubo_cfz_init_eta, enc_eta, enc_z, optimizer, Data, K, NUM_EPOCHS, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE, PATH, CUDA, DEVICE, RESAMPLE=RESAMPLE, DETACH=DETACH)

epoch=0, symKL=13393.713, EUBO=6302.468, ELBO=-7497.438, ESS=1.215 (23s)
epoch=1, symKL=10959.183, EUBO=5203.762, ELBO=-6163.640, ESS=1.217 (24s)
epoch=2, symKL=9137.110, EUBO=4379.717, ELBO=-5162.762, ESS=1.220 (24s)
epoch=3, symKL=7814.494, EUBO=3817.742, ELBO=-4405.618, ESS=1.227 (23s)
epoch=4, symKL=6742.078, EUBO=3353.655, ELBO=-3796.033, ESS=1.241 (25s)
epoch=5, symKL=5923.580, EUBO=3004.624, ELBO=-3330.532, ESS=1.252 (24s)
epoch=6, symKL=5250.435, EUBO=2696.971, ELBO=-2961.977, ESS=1.269 (21s)
epoch=7, symKL=4732.467, EUBO=2469.796, ELBO=-2673.611, ESS=1.287 (21s)
epoch=8, symKL=4309.627, EUBO=2261.822, ELBO=-2457.777, ESS=1.306 (21s)
epoch=9, symKL=3974.181, EUBO=2103.369, ELBO=-2282.754, ESS=1.328 (22s)
epoch=10, symKL=3689.424, EUBO=1948.831, ELBO=-2151.652, ESS=1.350 (21s)
epoch=11, symKL=3456.713, EUBO=1805.229, ELBO=-2058.920, ESS=1.371 (23s)
epoch=12, symKL=3249.216, EUBO=1664.817, ELBO=-1988.189, ESS=1.389 (23s)
epoch=13, symKL=3096.481, EUBO=1567.770, ELBO=-1938.836, ES

In [ ]:
# %debug

In [ ]:
# torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
RESAMPLE= False
obs, q_eta, q_z, symkl_test, eubo_test, elbo_test = test(Eubo_cfz_init_z, enc_eta, enc_z, Data, K, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, RESAMPLE=RESAMPLE)

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap